# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [112]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject
from HouseholdSpecializationModel import HouseholdSpecializationModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
model = HouseholdSpecializationModelClass()

# Question 1

In [125]:
#Unpaking model 
par = model.par


list_alpha = [0.25, 0.5, 0.75]
list_sigma = [0.5, 1, 1.5]

#Code solves the discrete model for lists of alpha and sigma 
for i in list_sigma:
    for j in list_alpha:
        par.sigma = i
        par.alpha = j
        opt = model.solve_discrete()
#        print(f' For sigma = {par.sigma:.2f} and alpha = {par.alpha:.2f} {opt}')
        
        relative_hours = opt.HF/opt.HM
        print(f' For sigma = {par.sigma:.2f} and alpha = {par.alpha:.2f}    relative hours at home: {relative_hours:.4f}')



 For sigma = 0.50 and alpha = 0.25    relative hours at home: 0.5455
 For sigma = 0.50 and alpha = 0.50    relative hours at home: 1.0000
 For sigma = 0.50 and alpha = 0.75    relative hours at home: 1.8333
 For sigma = 1.00 and alpha = 0.25    relative hours at home: 0.3077
 For sigma = 1.00 and alpha = 0.50    relative hours at home: 1.0000
 For sigma = 1.00 and alpha = 0.75    relative hours at home: 3.2500
 For sigma = 1.50 and alpha = 0.25    relative hours at home: 0.2143
 For sigma = 1.50 and alpha = 0.50    relative hours at home: 1.0000
 For sigma = 1.50 and alpha = 0.75    relative hours at home: 4.6667


# Question 2

**Ploting relative wage**

In [ ]:
list_wage = [] 